In [1]:
"""
1. Search for an open opportunity at your possible potential clients
2. Search for relevant information from the opening that might act as a good starting point for your business with this company
3. Go through all of your company's projects and find relevant projects
4. Get important to share information about these projects
5. Write a compelling cold email about how your firm can be of great service to your potential client
6. Repeat for ALL potential clients
"""

"\n1. Search for an open opportunity at your possible potential clients\n2. Search for relevant information from the opening that might act as a good starting point for your business with this company\n3. Go through all of your company's projects and find relevant projects\n4. Get important to share information about these projects\n5. Write a compelling cold email about how your firm can be of great service to your potential client\n6. Repeat for ALL potential clients\n"

In [2]:
# Web Scraping

# Relevant information

# Chromadb query

# Email Generation

In [12]:
# Web Scraping
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.dice.com/jobs/q-Software+Engineer-jobs")
page_data=loader.load().pop().page_content
print(page_data)

                        Software Engineer Jobs | Dice.com       Software Engineer JobsRefine Results1 - 20 of 47,383 JobsSenior Embedded Software EngineerJob Title - Senior Embedded Software EngineerSaveISCO International LLCCompany Name - ISCO International LLC•Schaumburg, Illinois, USAFull-timePosition Description  ISCO is looking for a Senior Embedded Software Engineer to help architect, develop and implement ISCO's new telecommunications products. This role has a wide scope that may include implementation of DSP algorithms, development of drivers and APIs, design of O&M functions, and even support of automated test development efforts. This role requires a Senior level Software Engineer with a broad range of skills, innovative spirit and desire to work in a fast-paced, entrepreneuriEasy ApplyPosted 2 hours ago Senior Software EngineerJob Title - Senior Software EngineerSaveFirst-Citizens Bank & Trust CompanyCompany Name - First-Citizens Bank & Trust Company•Phoenix, Arizona, USAFul

In [7]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, max_tokens=None, timeout=None, max_retries=2)


In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    I will give you scraped text from the job posting. 
    Your job is to extract the job details & requirements in a JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'. 
    Only return valid JSON. No preamble, please.
    Here is the scraped text: {page_data}
    """
)

chain_extract = prompt_extract | llm 
response = chain_extract.invoke(input={'page_data':page_data})
print(type(response.content))
print(response.content)

<class 'str'>
```json
[
  {
    "role": "Senior Embedded Software Engineer",
    "experience": "Senior level",
    "skills": "DSP algorithms, drivers, APIs, O&M functions, automated test development",
    "description": "Help architect, develop and implement ISCO's new telecommunications products"
  },
  {
    "role": "Senior Software Engineer",
    "experience": "Senior level",
    "skills": "Core banking, Payments & Money movement, technical guidance, estimation of efforts",
    "description": "Provide technology solutions and integration services"
  },
  {
    "role": "Staff Software Engineer, Vehicle Software",
    "experience": "Staff level",
    "skills": "Embedded software, drivers, operating system, BSP, software stack",
    "description": "Design, develop, and maintain the foundational embedded software platform for SDV solutions"
  },
  {
    "role": "Senior Software Engineer (AWS)",
    "experience": "Senior level",
    "skills": "AWS, innovation, collaboration",
    "descri

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
print(type(json_response))

<class 'list'>


In [19]:
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        for row in csv_reader:
            # Separate technical skills (list) and project link (string)
            skills = tuple(row[:-1])  # Exclude the last element (project link)
            project_link = row[-1]  # Get the last element (project link)
            data.append((skills, project_link))  # Create a tuple with skills and link
    return data

# Example usage:
file_path = 'sample_portfolio.csv'  # Replace with your actual file path
data = read_csv_file(file_path)

for skills, project_link in data:
    print(skills, project_link)
    #print(project_link)

('Python', ' SQL', ' Pandas')  https://github.com/user/project1
('SQL', ' Python', ' Airflow')  https://github.com/user/project2
('PySpark', ' Spark SQL', ' Delta Lake')  https://github.com/user/project3
('Machine Learning', ' Deep Learning', ' TensorFlow')  https://github.com/user/project4
('Data Engineering', ' ETL', ' ELT')  https://github.com/user/project5
('Cloud Platforms (AWS', ' GCP', ' Azure)')  https://github.com/user/project6
('Data Warehousing', ' Data Modeling', ' DBT')  https://github.com/user/project7
('Data Visualization', ' Power BI', ' Tableau')  https://github.com/user/project8
('MLOps', ' MLflow', ' Kubeflow')  https://github.com/user/project9
('Natural Language Processing (NLP)', ' NLTK', ' spaCy')  https://github.com/user/project10
('Computer Vision', ' OpenCV', ' TensorFlow')  https://github.com/user/project11
('Time Series Analysis', ' Forecasting', ' Prophet')  https://github.com/user/project12
('Data Cleaning', ' Data Wrangling', ' Pandas')  https://github.com

In [20]:
# Insert data into Vector Database

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio_links')

if not collection.count():
    for skills, project_url in data:
        collection.add(
            documents=str(skills),
            metadatas={'portfolio_url':project_url},
            ids=[str(uuid.uuid4())]
        )

In [ ]:

# all_skills_str = ', '.join([job['skills'] for job in json_response])
# print(all_skills_str)
# since there are many jobs in web scraping so we are doing testing of only one
first_job_skills = json_response[0]['skills']

In [34]:
portfolio_urls=collection.query(query_texts=first_job_skills,
                                n_results=2)
print(portfolio_urls)

{'ids': [['38bae0a5-2c1a-4cb8-8ab1-bee3e06bba3d', 'da4a5b16-6af4-468e-afd7-acadc1ea95df']], 'embeddings': None, 'documents': [["('DSP algorithms', ' Embedded C', ' APIs')", "('Compilers', ' C++', ' LLVM')"]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'portfolio_url': ' https://github.com/user/embedded-telecom  '}, {'portfolio_url': ' https://github.com/user/social-data-compiler  '}]], 'distances': [[0.5638918876647949, 1.1970778703689575]]}


In [35]:
first_job_description = json_response[0]['description']
print(first_job_description)

Help architect, develop and implement ISCO's new telecommunications products


In [36]:
email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your Role: Your name is Hassan, You are an incredible business development officer who knows how to get clients. You work for X Consulting firm, your firm works with all sorts of IT clients and provide solutions in the domain of Data Science and AI. 
    X AI focuses on efficient tailored solutions for all clients keeping costs down. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)

chain_email = email_prompt | llm
response = chain_email.invoke({'job_description':first_job_description, 'portfolio_urls':portfolio_urls})
print(response.content)

Subject: Expert Solutions for ISCO's Telecommunications Products Development

Dear Hiring Manager,

I came across the job posting for architecting, developing, and implementing ISCO's new telecommunications products, and I was impressed by the scope and ambition of the project. As a Business Development Officer at X Consulting firm, I believe our team can bring significant value to this initiative.

Our company, X AI, specializes in providing efficient, tailored solutions in the domain of Data Science and AI, with a strong focus on keeping costs down. We have a proven track record of delivering high-quality results in the telecommunications sector, and I'd like to highlight a few examples of our relevant work:

* Our expertise in DSP algorithms, Embedded C, and APIs can be seen in our portfolio project: https://github.com/user/embedded-telecom
* Additionally, our experience with Compilers, C++, and LLVM is showcased in another project: https://github.com/user/social-data-compiler

Thes